In [1]:
# Peter Mansbacher
# Instagram scraper
# used code cited throughout program

In [2]:
from bs4 import BeautifulSoup ##html parsing library
import bs4 
import pandas as pd #dataframe library
import numpy as np #libary for math computations
import selenium.webdriver as webdriver#web browser automating
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import WebDriverException
from webdriver_manager.chrome import ChromeDriverManager
import requests#http library
from time import sleep#time module
import random#random libary
from datetime import date#date handling
import datetime
import pymysql

In [3]:
brand1='xxl'
brand2='Apple'
brand3='LGUSA'
date2=date.today()
date1= date2-datetime.timedelta(5)
list_ = [date2-datetime.timedelta(1),date2-datetime.timedelta(2),date2-datetime.timedelta(3),date2-datetime.timedelta(3),date2-datetime.timedelta(4),date2-datetime.timedelta(5),date2-datetime.timedelta(6)]
saved=[]


In [4]:
def scraper(brand1,date1,date2):
    url = "https://www.instagram.com/" + brand1
    chrome_options = Options()
    chrome_options.add_argument("start-maximized")
    chrome_options.add_argument("disable-infobars")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-setuid-sandbox")
    user="CSCI4738"
    password_="Password4738"
    browser = webdriver.Chrome(options=chrome_options, executable_path=ChromeDriverManager().install())
    browser.get('https://www.instagram.com/')
    print(browser.title)
    sleep(random.randint(30,55))
    username = browser.find_element_by_name('username')#create object to hold username and edit interact with html code
    username.clear()#clear username
    sleep(5)
    username.send_keys(user)#set username
    sleep(5)
    password= browser.find_element_by_name('password')#create password obj
    password.clear()#clear field
    sleep(7)
    password.send_keys(password_)#send password
    sleep(6)
    submit=browser.find_element_by_tag_name('form')#submit obj 
    submit.submit()#enter
    sleep(random.randint(30,55))
    notsaved=browser.find_element_by_class_name("yWX7d")#for not saving password
    notsaved.click()
    sleep(random.randint(15,30))
    notnow=browser.find_element_by_class_name("HoLwm")#for not saving password
    notnow.click()
    sleep(random.randint(15,30))
    browser.get(url)
    sleep(random.randint(15,30))
    elem = browser.find_element_by_tag_name("body")
    no_of_pagedowns = 2
    page_source = browser.page_source
    page = BeautifulSoup(page_source, 'html.parser')
    divs = page.find_all('div')
    urls=[]
    thumbnails = []
    tags= []
    dateList = []
    #titles change between ' likes' and 'views', but always remain in same section tag with class 'EDfFK ygqzn'
    views = []
    likes = []
    captions = []
    capString = ""
    while no_of_pagedowns:
        divs += page.find_all('div')
        #https://medium.com/analytics-vidhya/using-python-and-selenium-to-scrape-infinite-scroll-web-pages-825d12c24ec7
        #for parent in page.find_all(class_="Nnq7C weEfm"):
        for parent in page.find_all(class_="v1Nh3 kIKUG _bz0w"):
                #find a tag to get post links from href
                a_tag = parent.find("a")
                #href for post links
                link = a_tag.attrs['href']
                #img tag for post thumbnail in case date in range
                alt_tag = parent.find('img')
                print('www.instagram.com'+link)
                browser.get('https://www.instagram.com'+link)
                page_source = browser.page_source
                soup = BeautifulSoup(page_source, 'html.parser')
                #get dates########################################
                dateTag=soup.find('time')
                dateFull = dateTag.attrs['datetime']
                print('date={}'.format(dateFull[0:10]))
                print()
                year, month, day = map(int, dateFull[0:10].split('-'))
                dateFull = datetime.date(year,month,day)
                print(dateFull)
                if(date1<=dateFull<=date2):
                    #date in range so we can append post link and post thumbnail link to df
                    date=""
                    urls.append('https://www.instagram.com' + link)#continue puttings post links in url list
                    thumbnails.append(alt_tag.attrs['src'])
                    dateList.append(dateFull)
                    print("date={}".format(dateFull))
                    #get caption#####################################
                    capString = ""
                    caption = browser.title
                    captionStrip =caption.strip()
                    length = len(captionStrip)
                    for i in range(0,length):
                        if (i>= 20 and i<=length):
                            #print(captionStrip[i], end='')
                            capString+=captionStrip[i]
                    captions.append(capString)
                    #get likes or views amount#######################
                    sad = soup.find(class_='EDfFK ygqzn').get_text()
                    print(sad)
                    split = sad.split()
                    if (split[1]=='likes'):
                        #print('/n*likes/n')
                        likes.append(split[0])
                        split[0]=split[0].replace(',', '')
                        liked=int(split[0])
                        views_=0
                        views.append(0)
                    elif(split[1]=='views'):
                        views.append(split[0])
                        split[0]=split[0].replace(',', '')
                        views_=int(split[0])
                        liked=0
                        #print('/n*views/n')
                        likes.append(0)
                    # ESTABLISH CONNECTION WITH DB
                    server = "ls-1ef1825172e62dcc237ee491d09a0c12aff562fe.cn5ycdfnko6g.us-east-1.rds.amazonaws.com"
                    database_ = 'smcDB'
                    username = 'dbmasteruser'
                    password_ = 'q+o.H1sd$CRRZl&CSl>VK}-(~+t1ea&P'
                    db = pymysql.connect(host=server, user=username, password=password_, database=database_, charset='utf8mb4',
                    cursorclass=pymysql.cursors.DictCursor, port=3306)
                    cursor = db.cursor()
                    sql = '''
                     insert into smcDB.INSTAGRAM(BrandHandle,PostUrl,PostDate,PostText,Likes,ImageUrl,Views) values('%s','%s',
                     '%s','%s','%i','%s','%i') 
                     ''' % (brand1, 'https://www.instagram.com' + link, dateFull, capString, liked, alt_tag.attrs['src'], views_)
                    try:#Andres try and except statement
                        cursor.execute(sql)
                        db.commit()
                        print("Successfully committed to database")
                    except:
                        print("Error commiting to database or already in database")

         #        elem.send_keys(Keys.PAGE_DOWN)
        sleep(random.randint(15,20))
        no_of_pagedowns=0

    df = pd.DataFrame(columns=['BrandHandle','PostUrl','PostDate','PostText','Likes','Comments', 'ImageUrl',' Views', 'Likes'])
    df['BrandHandle']=brand1
    df['PostUrl'] = urls
    df['PostDate'] = dateList
    df['Comments'] = captions
    df['ImageUrl'] = thumbnails
    df['Views']= views
    df['Likes']= likes
    #print("brand1={}\nposturl={}\ndate={}\ncomment={}\nthumbnail={}\nviews={}".format(type(brand1[0]),type(urls[0]),type(dateList[0]), type(captions[0]),type(thumbnails[0]),type(views[0]),type(likes[0])))
    print("brand1={}\nposturl={}\ndate={}\ncomment={}\nthumbnail={}\nviews={}".format(len(brand1),len(urls),len(dateList), len(captions),len(thumbnails),len(views),len(likes)))



            
            
        

 
    
    




In [ ]:
1

In [8]:
df = scraper(brand1,date1,date2)



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST driver version for 99.0.4844
Driver [C:\Users\peter\.wdm\drivers\chromedriver\win32\99.0.4844.51\chromedriver.exe] found in cache
C:\Users\peter\AppData\Local\Temp/ipykernel_5756/1979085319.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(options=chrome_options, executable_path=ChromeDriverManager().install())


Instagram


C:\Users\peter\AppData\Local\Temp/ipykernel_5756/1979085319.py:16: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  username = browser.find_element_by_name('username')#create object to hold username and edit interact with html code
C:\Users\peter\AppData\Local\Temp/ipykernel_5756/1979085319.py:21: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  password= browser.find_element_by_name('password')#create password obj
C:\Users\peter\AppData\Local\Temp/ipykernel_5756/1979085319.py:26: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  submit=browser.find_element_by_tag_name('form')#submit obj


KeyboardInterrupt: 

In [91]:
type(df)

NoneType